# Diamond Flow Example

This notebook demonstrates a branching and merging flow pattern:

```
       A
      / \
     B   C
      \ /
       D
```

Key concepts demonstrated:
- **Branching**: Node A sends packets to both B and C
- **Merging**: Node D waits for packets from both B and C
- **Synchronization**: D's epoch only triggers when **both** inputs are present

## Setup

In [43]:
from netrun_sim import (
    # Graph types
    Graph, Node, Edge, Port, PortRef, PortType, PortSlotSpec,
    PortState, SalvoCondition, SalvoConditionTerm,
    # Net types
    Net, NetAction, NetActionResponseData, PacketLocation,
)

## Helper Functions

In [44]:
def create_edge(src_node: str, src_port: str, tgt_node: str, tgt_port: str) -> Edge:
    """Create an edge between two ports."""
    return Edge(
        PortRef(src_node, PortType.Output, src_port),
        PortRef(tgt_node, PortType.Input, tgt_port),
    )


def edge_location(src_node: str, src_port: str, tgt_node: str, tgt_port: str) -> PacketLocation:
    """Create a PacketLocation for an edge."""
    return PacketLocation.edge(
        Edge(
            PortRef(src_node, PortType.Output, src_port),
            PortRef(tgt_node, PortType.Input, tgt_port),
        )
    )

## Creating the Nodes

### Node A: Source with Two Outputs

Node A is the source - it has two output ports (`out1` and `out2`) that will send packets to B and C respectively.

In [45]:
node_a = Node(
    name="A",
    out_ports={
        "out1": Port(PortSlotSpec.infinite()),
        "out2": Port(PortSlotSpec.infinite()),
    },
)

print("Node A: source with outputs 'out1' (→ B) and 'out2' (→ C)")

Node A: source with outputs 'out1' (→ B) and 'out2' (→ C)


### Nodes B and C: Middle Nodes

B and C are simple pass-through nodes. Each has one input and one output.

In [46]:
def create_simple_node(name: str) -> Node:
    """Create a simple node with one input and one output."""
    return Node(
        name=name,
        in_ports={"in": Port(PortSlotSpec.infinite())},
        out_ports={"out": Port(PortSlotSpec.infinite())},
        in_salvo_conditions={
            "default": SalvoCondition(
                max_salvos=1,
                ports=["in"],
                term=SalvoConditionTerm.port("in", PortState.non_empty()),
            ),
        },
        out_salvo_conditions={
            "default": SalvoCondition(
                max_salvos=0,
                ports=["out"],
                term=SalvoConditionTerm.port("out", PortState.non_empty()),
            ),
        },
    )


node_b = create_simple_node("B")
node_c = create_simple_node("C")

print("Node B: middle node (receives from A.out1, sends to D.in1)")
print("Node C: middle node (receives from A.out2, sends to D.in2)")

Node B: middle node (receives from A.out1, sends to D.in1)
Node C: middle node (receives from A.out2, sends to D.in2)


### Node D: Synchronizing Merge Node

Node D has **two input ports** (`in1` from B, `in2` from C). Its salvo condition uses an **AND** term - it requires packets on **both** inputs before triggering an epoch.

This demonstrates synchronization: D waits for both branches to complete before processing.

In [47]:
node_d = Node(
    name="D",
    in_ports={
        "in1": Port(PortSlotSpec.infinite()),
        "in2": Port(PortSlotSpec.infinite()),
    },
    in_salvo_conditions={
        "default": SalvoCondition(
            max_salvos=1,
            ports=["in1", "in2"],
            # Require BOTH inputs to be non-empty (synchronization!)
            term=SalvoConditionTerm.and_([
                SalvoConditionTerm.port("in1", PortState.non_empty()),
                SalvoConditionTerm.port("in2", PortState.non_empty()),
            ]),
        ),
    },
)

print("Node D: merge node with TWO inputs")
print("  - in1: receives from B")
print("  - in2: receives from C")
print("  - Triggers only when BOTH inputs have packets (AND condition)")

Node D: merge node with TWO inputs
  - in1: receives from B
  - in2: receives from C
  - Triggers only when BOTH inputs have packets (AND condition)


## Creating the Graph

Now let's connect everything:
- A.out1 → B.in
- A.out2 → C.in
- B.out → D.in1
- C.out → D.in2

In [48]:
edges = [
    create_edge("A", "out1", "B", "in"),
    create_edge("A", "out2", "C", "in"),
    create_edge("B", "out", "D", "in1"),
    create_edge("C", "out", "D", "in2"),
]

graph = Graph([node_a, node_b, node_c, node_d], edges)

errors = graph.validate()
assert len(errors) == 0, f"Validation errors: {errors}"

print("Diamond graph created and validated!")
print(f"Nodes: {list(graph.nodes().keys())}")
print(f"Edges: {len(graph.edges())}")

Diamond graph created and validated!
Nodes: ['B', 'A', 'D', 'C']
Edges: 4


## Creating the Network and Packets

We'll create two packets - one for each branch of the diamond.

In [49]:
net = Net(graph)

# Create two packets
response1, _ = net.do_action(NetAction.create_packet())
response2, _ = net.do_action(NetAction.create_packet())

assert isinstance(response1, NetActionResponseData.Packet)
assert isinstance(response2, NetActionResponseData.Packet)

packet1 = response1.packet_id
packet2 = response2.packet_id

print(f"Created packet 1: {packet1}")
print(f"Created packet 2: {packet2}")

Created packet 1: 01KDBPE7HY1CHWWBEW5YA5VFZ3
Created packet 2: 01KDBPE7HYJ61049P52PD9W5ZW


## Placing Packets on Edges from A

Place each packet on a different branch:
- Packet 1 → edge A.out1 → B
- Packet 2 → edge A.out2 → C

In [50]:
# Place packet1 on edge A → B
net.do_action(
    NetAction.transport_packet_to_location(
        packet1, 
        edge_location("A", "out1", "B", "in")
    )
)
print("Placed packet 1 on edge A → B")

# Place packet2 on edge A → C
net.do_action(
    NetAction.transport_packet_to_location(
        packet2, 
        edge_location("A", "out2", "C", "in")
    )
)
print("Placed packet 2 on edge A → C")

Placed packet 1 on edge A → B
Placed packet 2 on edge A → C


## First Run: Packets Move to B and C

Running the network will move packets from edges to input ports, triggering epochs at both B and C.

In [51]:
net.do_action(NetAction.run_net_until_blocked())

startable = net.get_startable_epochs()
print(f"Startable epochs: {len(startable)}")

for epoch_id in startable:
    epoch = net.get_epoch(epoch_id)
    print(f"  - Epoch at node {epoch.node_name}")

Startable epochs: 2
  - Epoch at node B
  - Epoch at node C


## Processing B and C in Parallel

Now we process both B and C. Each will:
1. Start its epoch
2. Consume the input packet
3. Create an output packet
4. Send it to D

In [52]:
def process_node(net: Net, epoch_id) -> None:
    """Process a simple pass-through node."""
    # Start the epoch
    response, _ = net.do_action(NetAction.start_epoch(epoch_id))
    assert isinstance(response, NetActionResponseData.StartedEpoch)
    epoch = response.epoch
    print(f"\nProcessing node {epoch.node_name}:")
    
    # Consume input packet
    input_packet = epoch.in_salvo.packets[0][1]
    net.do_action(NetAction.consume_packet(input_packet))
    print(f"  Consumed input packet")
    
    # Create output packet
    response, _ = net.do_action(NetAction.create_packet(epoch.id))
    assert isinstance(response, NetActionResponseData.Packet)
    output_packet = response.packet_id
    print(f"  Created output packet")
    
    # Load and send
    net.do_action(NetAction.load_packet_into_output_port(output_packet, "out"))
    net.do_action(NetAction.send_output_salvo(epoch.id, "default"))
    print(f"  Sent output to downstream edge")
    
    # Finish
    net.do_action(NetAction.finish_epoch(epoch.id))
    print(f"  Finished epoch")

In [53]:
# Process both B and C
for epoch_id in startable:
    process_node(net, epoch_id)


Processing node B:
  Consumed input packet
  Created output packet
  Sent output to downstream edge
  Finished epoch

Processing node C:
  Consumed input packet
  Created output packet
  Sent output to downstream edge
  Finished epoch


## Second Run: Packets Move to D

Now let's run the network again. The packets from B and C will move to D's input ports.

In [54]:
net.do_action(NetAction.run_net_until_blocked())

# Check D's input ports
d_in1 = PacketLocation.input_port("D", "in1")
d_in2 = PacketLocation.input_port("D", "in2")

print("D's input ports after second run:")
print(f"  in1 (from B): {net.packet_count_at(d_in1)} packet(s)")
print(f"  in2 (from C): {net.packet_count_at(d_in2)} packet(s)")

D's input ports after second run:
  in1 (from B): 0 packet(s)
  in2 (from C): 0 packet(s)


## Synchronization: D's Epoch is Ready

Since D requires **both** inputs to be non-empty (AND condition), it should now have a startable epoch.

In [55]:
startable_d = net.get_startable_epochs()
print(f"Startable epochs at D: {len(startable_d)}")

if startable_d:
    d_epoch = net.get_epoch(startable_d[0])
    print(f"\nEpoch details:")
    print(f"  Node: {d_epoch.node_name}")
    print(f"  Packets in input salvo: {len(d_epoch.in_salvo.packets)}")
    
    for port_name, packet_id in d_epoch.in_salvo.packets:
        print(f"    - Port '{port_name}': packet {packet_id[:12]}...")

Startable epochs at D: 1

Epoch details:
  Node: D
  Packets in input salvo: 2
    - Port 'in1': packet 01KDBPE7JFPW...
    - Port 'in2': packet 01KDBPE7JFNA...


## Summary

We successfully demonstrated a diamond flow pattern:

1. **Branching**: Two packets were sent from A on different paths (A→B and A→C)
2. **Parallel Processing**: B and C processed their packets independently
3. **Synchronization**: D waited until **both** branches completed
4. **Merging**: D's epoch received packets from both B and C

The key insight is D's salvo condition using `SalvoConditionTerm.and_()` to require both inputs. This is a fundamental pattern for coordinating parallel work in flow-based systems.

In [56]:
print("Diamond flow example complete!")

Diamond flow example complete!
